> Building llama.cpp and running a Mistral-v0.1 model.

# Intro  

This notebook runs an LLM using the [llama.cpp](https://github.com/ggerganov/llama.cpp) library. Specifically we run the powerful, recently released [`Mistral-7B-Instruct-v0.1`](https://mistral.ai/news/announcing-mistral-7b/) model.


# llama.cpp

llama.cpp is designed to run quantized LLMs on a Mac. Despite its name, the project supports many other models beyond Llama and Llama-2. There are even [python bindings](https://github.com/abetlen/llama-cpp-python) to make our lives easier.  

The picture below comes from project's README, and shows low-level details about how the repo works and what it supports. 

![](llama_description.png)

llama.cpp was originally hacked together in a [single evening](https://github.com/ggerganov/llama.cpp/issues/33#issuecomment-1465108022), and has since become arguably the SOTA for deploying LLMs on CPUs. This is in large part thanks to the incredibly helpful and responsive community behind it.  

Below we can see the full list of models that llama.cpp supports as of writing.  

![](llama_model_support.png)

The benefits of llama.cpp go beyond its code or models. Folks are always collaborating in [Pull Requests](https://github.com/ggerganov/llama.cpp/pulls) to bring in the latest, greatest advances from the flood of LLM progress. Tracking these PRs is a great way of keeping up to date with the field. Thankfully, the community is very open to hackers and new ideas: if something works and there's proof, then it gets merged in.  

Next, let's use llama.cpp to run a `Mistral-v0.1` model.


# Running Mistral-v0.1 with llama.cpp

In this section we cover the following:  
- Installing the `llama.cpp` repo  
- Downloading a `Mistral-v0.1` model  
- Running the model directly with `llama.cpp`    
- Running the model in a Jupyter Notebook  

First, we create a mamba environment to keep our work isolated. Then we download and install the repo.

Next we download the actual Mistral model from the HuggingFace Model Hub.

Lastly, we run the Mistral model on a sample input.

### Building llama.cpp

Let's get started. First create a new python3.11 mamba environment:

```bash
# create an environment for llama.cpp
mamba create -n llama-cpp python=3.11
```

This isn't *strictly* necessary for llama.cpp since it uses C++. But we will need it later on for the python bindings. And in any case, it's best practice to keep our projects in isolated environments. 

Next activate the new environment: 

```bash
# activate the environment
mamba activate llama-cpp
```

Then go ahead and clone the repo. Once it's cloned, we can move inside it and prepare for the build.

```bash 
# clone and move into the llama.cpp repo
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
```

There are two options to build llama.cpp:  
- [GNU Make](https://www.gnu.org/software/make/)  
- [CMake](https://cmake.org/)    

`make` works great on Linux, but I've had mixed results on Mac. For that reason we'll stick with `CMake` instead.   

The safest bet is to grab the CMake installer from the [official site](https://cmake.org/download/) and run it. But any reasonable package manager for Mac and Linux (e.g. `brew` or `apt`) should also work.

With CMake installed we can now follow a standard build process. Start by creating and moving into a special `build/` folder:

```bash
# create a build directory and move into it
mkdir build
cd build
```

Then run the `cmake` command to prepare the build. We can also specify special build options. For example, on Mac we can pass the `LLAMA_METAL=1` flag to use the GPU, or on Linux we can pass the `LLAMA_CUBLAS=1` flag to use an NVIDIA GPU.

```bash
# prepare the llama.cpp build with Mac hardware acceleration
cmake -DLLAMA_METAL=1 ..

# # or, use the line below on Linux/Window to build for NVIDIA GPUs
# cmake -DLLAMA_CUBLAS=1 ..
```


With the setup files ready, we can now build the project with the command below:  
```bash 
# build the accelerated llama.cpp project
cmake --build . --config Release 
```

Once the build finishes, the output binaries will be inside the `build/bin` folder. This folder has an executable binary file called `main`. `main` is how we'll be calling llama.cpp to run LLMs.

We are now ready to grab the Mistral model.

## Downloading a Mistral-v0.1 model

We will run a `Mistral-7B-Instruct-v0.1` model. What exactly does its name mean? Let's breakdown it down a bit:
- `Mistral` is the name given by the developers, in this case the [Mistral.ai team](https://mistral.ai/) 
- `7B` means that the model has 7 billion parameters   
- `Instruct` means that it was trained to follow and complete user instructions  
- `v0.1` is the release version for this model   

Below is the link to the model in the HuggingFace Hub

> HuggingFace link to [`Mistral-7B-Instruct-v0.1`](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF)

Follow the link above, then click on the `Files and version` tab near the top. You'll see a list of models that were quantized in different ways:

![](mistral_quantized.png)

These names can be overwhelming. Let's break them down at a high level. Feel free to skim the next couple of paragraphs, they are not critical to running the model itself. 

You can see that each file ends with a format like this: `Q*_*.gguf`. For example one model from the list is: `mistral-7B-Instruct-v0.1.Q4_K_S.gguf`. The `Q4` part means that the model was quantized with 4-bits. The `K_S` part refers to the specific flavor of quantization that was used.  

There is an unfortunate tradeoff between quantization and performance. The fewer bits we use, the smaller and faster the model will be but the worse its performance. And the more bits we use, the better its performance but the slower the model. In general, the `Q4` and `Q5` models offer a good balance between speed, performance, and size. Now let's get back to running the model. 

We choose the `Q5_K_M` model which is a bit larger than the `Q4` models but makes up for it in performance.

To grab this model, first make sure the huggingface-hub CLI is installed:

```bash
# install a tool to download HuggingFace models via the terminal
pip install huggingface-hub
```  

Then move into the `models/` folder inside of the llama.cpp repo and download the model with the following command:

```bash
# download the Mistral Q5_K_M model
huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF \
    mistral-7b-instruct-v0.1.Q5_K_M.gguf \
    --local-dir . \
    --local-dir-use-symlinks False
```

Once the model has downloaded, we are ready to run it.

```bash
# from build/, run the official example to see Mistral-v0.1 in action
./bin/main -m ../models/mistral-7b-instruct-v0.1.Q5_K_M.gguf \
    -p "Building a website can be done in 10 simple steps:\nStep 1:"
```

## Running the Mistral model

We'll use the `main` binary inside of the `build/` folder from before to run the `Q5_K_M` model. 

Run the following command to see the Mistral LLM in action! Here we ask to tell us how to build a website in 10 steps.

The `-m` flag points to the model file, which we stored under `models/`. The `-p` flag is the prompt for the model to follow.

Here's a short snippet from my output after running the command:

```
Building a website can be done in 10 simple steps:

Step 1: Choose a domain name. A domain name is the unique address of your website on the internet, which is what people will use to find it. Consider choosing a name that reflects the purpose or brand of your website.

Step 2: Choose a hosting provider. Your website needs to be hosted by a server so that it can be accessed by others on the internet. Choose a reliable hosting provider that suits your budget and technical needs.

Step 3: Install a content management system (CMS). A CMS is software that allows you to create, manage and publish content on your website without needing to know how to code. Examples of popular CMS platforms include WordPress, Drupal, and Joomla.
```

And that's it! We have now done the following:  
- Installed llama.cpp.  
- Downloaded a Mistral-v0.1 model.  
- Ran the Mistral model on a sample input.

Everything so far was done in C++. Next we run the Mistral model inside a Jupyter Notebook with the llama.cpp python bindings.

# Running Mistral-v0.1 with python

Start by installing the llama.cpp python bindings in the mamba environment.

Here are the instructions for the full [Mac installation](https://github.com/abetlen/llama-cpp-python/blob/main/docs/install/macos.md). The pair of pip commands below will install the bindings.  

```bash
# install the python bindings with Metal acceleration
pip uninstall llama-cpp-python -y
CMAKE_ARGS="-DLLAMA_METAL=on" pip install -U llama-cpp-python --no-cache-dir
```  

The command above does the following to make sure the python bindings are up to date:  
- Uninstalls older versions of the bindings, if any are found.    
- Installs the bindings with Metal (Mac GPU) acceleration.

After installing the bindings, run the following code snippet. This will check if the bindings were installed correctly. 

In [1]:
# check if we can import the llama.cpp python bindings 
from llama_cpp import Llama

If the command above works, we can now run the Mistral-v0.1 model inside a Jupyter Notebook. We instantiate the `Llama` class by pointing it to the model weights we downloaded earlier. Make sure to change the paths to match your own.

In [2]:
#|output: false
# point the Llama class to the model weights we downloaded in the previous sections
work_dir = "/Users/cck/repos/llama.cpp/"
llm = Llama(f"{work_dir}/models/mistral-7b-instruct-v0.1.Q5_K_M.gguf");

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/cck/repos/llama.cpp//models/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.we

In [3]:
#|output: false
# asking Mistral to help us build a website
prompt = "Building a website can be done in 10 simple steps:\nStep 1:"
output = llm(prompt, max_tokens=512, echo=True);


llama_print_timings:        load time =  6055.88 ms
llama_print_timings:      sample time =   313.45 ms /   462 runs   (    0.68 ms per token,  1473.91 tokens per second)
llama_print_timings: prompt eval time =  6055.79 ms /    19 tokens (  318.73 ms per token,     3.14 tokens per second)
llama_print_timings:        eval time = 27402.22 ms /   461 runs   (   59.44 ms per token,    16.82 tokens per second)
llama_print_timings:       total time = 34361.83 ms


In [32]:
#| code-overflow: wrap
output['choices'][0]['text']

'Building a website can be done in 10 simple steps:\nStep 1: Define the Purpose of Your Website\nBefore you start building your website, it’s important to determine its primary purpose. This may include selling products or services, promoting a business or organization, providing information, or entertaining visitors.\nStep 2: Choose Your Web Design\nNext, decide on the design and layout of your website. This can include selecting a template or creating a custom design. Consider using a responsive design that adjusts to different screen sizes.\nStep 3: Purchase a Domain Name\nChoose a domain name that is easy to remember and reflects your brand. Register it with a domain registrar like GoDaddy or Namecheap.\nStep 4: Choose a Hosting Provider\nA web host provider will store your website files and make them available to visitors. Some popular options include Bluehost, HostGator, and DreamHost.\nStep 5: Create Your Website Content\nCreate the content for your website, including text, imag

Congrats! We've now ran the `Mistral-7B-Instruct-v0.1` model with llama.cpp in both C++ and python. The C++ version is ideal for a server or production application. And as for python version, we can now bootup a handy LLM assistant in a Jupyter Notebook, and ask it questions as we code or develop. 

## Conclusion

This notebook covered the llama.cpp library and how to use it to run LLMs. We then ran a `Mistral-7B-Instruct-v0.1` model with llama.cpp in both C++ and python.